In [64]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from datetime import datetime, timedelta

In [96]:
last_known_date = datetime.now().strftime('%Y-%m-%d')
def jama_get_extra_info(doi):
    global last_known_date
    response = requests.get(f'https://api.crossref.org/works/{doi}')
    json_resp = response.json()
    abstract = BeautifulSoup(json_resp['message']['abstract'],'html.parser').text
    title = json_resp['message']['title'][0]
    subjects = json_resp['message']['subject']
    url = json_resp['message']['link'][0]['URL']
    try:
        date = '-'.join(str(item) for item in json_resp['message']['published-print']['date-parts'][0])
        last_known_date = date
    except:
        date = last_known_date
    source = 'Jama'
    authors = []
    affiliations = []
    for id in json_resp['message']['author']:
        name = ""
        for name_type in ['given', 'family']:
            try:
                name+=id[name_type]+' '
            except:
                pass
        authors.append(name[:-1])
        for affil in id['affiliation']:
            affiliations.append(affil['name'])
    affiliations = list(set(affiliations))
    authors = list(set(authors))
    return abstract, title, subjects, url, date, source, authors, affiliations

def jama_get_days_back(days_back):
    days_back = 5#find and remove later
    out = []
    i=1
    while True:
        urlString = f'https://jamanetwork.com/searchresults?sort=Newest&f_ArticleTypeDisplayName=Research&page={i}'
        i+=1
        driver = webdriver.Chrome()
        driver.get(urlString)
        html = driver.page_source
        driver.close()
        soup=BeautifulSoup(html,'html.parser')
        for cite in soup.find_all('cite', class_ ='article--citation'):
            paper_info = {}
            paper_info['doi'] = cite.text.replace('. ', ';').split(';')[-1].strip()
            try:
                paper_info['abstract'],paper_info['title'],paper_info['subjects'],paper_info['url'],paper_info['date'],paper_info['source'],paper_info['authors'],paper_info['affiliations'] = jama_get_extra_info(paper_info['doi'])
            except:
                continue
            out.append(paper_info)
        if len(out)>0:
            if datetime.strptime(out[-1]['date'], '%Y-%m-%d').date() <datetime.now().date()-timedelta(days=days_back):
                break
    return pd.DataFrame.from_records(out)

In [ ]:
jama_get_extra_info('10.1001/jamapsychiatry.2023.3708')

('ImportanceThe reported prevalence of autism in children has consistently risen over the past 20 years. The concurrent implications for the adult Medicaid system, which insures autistic adults due to low income or disability, have not been studied.ObjectiveTo estimate the prevalence of adults identified as autistic in Medicaid claims data and to examine the prevalence by year, age, and race and ethnicity to understand enrollment patterns.Design, Setting, and ParticipantsThis cohort study used data from a longitudinal Medicaid claims cohort of enrollees aged 18 years or older with a claim for autism at any point from January 1, 2011, to December 31, 2019, and an approximately 1% random sample of all adult Medicaid enrollees. The data were analyzed between February 22 and June 22, 2023.ExposureAdults enrolled in Medicaid with a claim for autism.Main Outcome and MeasuresPrevalence of autism per 1000 Medicaid enrollees for each year was calculated using denominator data from the Centers f